# Extending GALINI

This Jupyter notebook shows how to extending GALINI using Python.

## Internal model representation

GALINI uses Pyomo to represent models internally. GALINI supports models where common subexpressions are grouped together to form a _connected model_. You can create a connected model using [SUSPECT](https://github.com/cog-imperial/suspect).

Connected models can help find better bounds when performing feasibility based bounds tightening (FBBT) and optimization based bounds tightening (OBBT) and can also help finding better convexity information.

In [2]:
import pyomo.environ as pe

def create_example_model():
    """Create a simple nonlinear model that will be used in the notebook."""
    m = pe.ConcreteModel()

    # Add (bounded) continuous variables
    m.y = pe.Var(bounds=(0, 2.0))
    m.z = pe.Var(bounds=(0, 3.0))

    # Add linear objective
    m.cost = pe.Objective(expr=m.y + m.z)

    # Add nonlinear constraints
    m.sq = pe.Constraint(expr=m.y**2 <= 0.6)
    m.bi = pe.Constraint(expr=m.y * m.z <= 1.5)
    m.bi_lb = pe.Constraint(expr=m.y * m.z >= 1.0)

    return m

In [3]:
# Create and print model
model = create_example_model()
model.pprint()

2 Var Declarations
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :   2.0 : False :  True :  Reals
    z : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :   3.0 : False :  True :  Reals

1 Objective Declarations
    cost : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : y + z

3 Constraint Declarations
    bi : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :  y*z :   1.5 :   True
    bi_lb : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :   1.0 :  y*z :  +Inf :   True
    sq : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf : y**2 :   0.6 :   True

6 Declarations: y z cost sq bi bi_lb


The connected model looks exactly the same as the original model, but the expression `x*y` is shared between the constraints `bi` and `bi_lb`.

In [6]:
from suspect.pyomo import create_connected_model

connected_model, _ = create_connected_model(model)

connected_model.pprint()

2 Var Declarations
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :   2.0 : False :  True :  Reals
    z : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :   3.0 : False :  True :  Reals

1 Objective Declarations
    cost : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : y + z

3 Constraint Declarations
    bi : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :  y*z :   1.5 :   True
    bi_lb : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :   1.0 :  y*z :  +Inf :   True
    sq : Size=1, Index=None, Active=True
        Key  : Lower : Body   : Upper : Active
        None :  -Inf : y**2.0 :   0.6 :   True

6 Declarations: y z cost sq bi bi_lb


## Building Relaxations

GALINI uses [Coramin](https://github.com/coramin/coramin) to compute relaxations of Pyomo models. Coramin works by replacing nonlinear expressions with auxiliary variables and using Pyomo blocks to create dynamic under and over estimators of the nonlinear expressions.

#### Coramin Relaxation Block

In this example we show how to create a McCormick envelope of the expression $w=xy$.


In [7]:
from coramin.relaxations.mccormick import PWMcCormickRelaxation

relax_example = pe.ConcreteModel()
relax_example.w = pe.Var()
relax_example.x = pe.Var(bounds=(-1, 1))
relax_example.y = pe.Var(bounds=(-2, 2))
relax_example.mccormick = PWMcCormickRelaxation()
relax_example.mccormick.set_input(
    x1=relax_example.x,
    x2=relax_example.y,
    aux_var=relax_example.w
)
relax_example.mccormick.rebuild()
relax_example.mccormick.pprint(verbose=True)

mccormick: Relaxation for w == x*y
  mccormick : Size=1, Index=None, Active=True
      1 Set Declarations
          relaxation_index : Size=1, Index=None, Ordered=Insertion
              Key  : Dimen : Domain : Size : Members
              None :     1 :    Any :    4 : {1, 2, 3, 4}

      1 Constraint Declarations
          relaxation : Size=4, Index=mccormick.relaxation_index, Active=True
              Key : Lower : Body              : Upper : Active
                1 :  -Inf : - y - 2*x - 2 - w :   0.0 :   True
                2 :  -Inf :   y + 2*x - 2 - w :   0.0 :   True
                3 :  -Inf : w - (y - 2*x + 2) :   0.0 :   True
                4 :  -Inf : w - (2*x - y + 2) :   0.0 :   True

      2 Declarations: relaxation_index relaxation


##### How to create a new relaxation

## Branching Strategy

## Node Selection Strategy

## Cut Generators

## Branch and Bound Algorithm